In [25]:
from bs4 import BeautifulSoup
import requests
import time
import glob
import re
import pandas as pd

# scraping NDAR for likely data sharing targets
In this notebook, we go through the 'collections' and 'studies' on NDAR and download any non-blank html pages. We then go through those pages and pull out any DOIs listed on those pages.

In [38]:
missing_ids = []
print_statuses = list(range(4200, 5000, 100))
for i in range(4203, 5000):
    if i in print_statuses:
        print(i)
    root_link = 'https://nda.nih.gov/edit_collection.html?id='
    lnk = root_link + str(i)
    r = requests.get(lnk)
    if len(r.history)==0:
        with open('output/ndar_collections/collection_'+str(i)+'.html', 'wb') as f:
            f.write(r.content)
    else:
        missing_ids.append(i)
    time.sleep(.25)

4300
4400
4500
4600
4700
4800
4900


In [39]:
missing_ids = []
print_statuses = list(range(200, 800, 50))
for i in range(200, 800):
    if i in print_statuses:
        print(i)
    root_link = 'https://nda.nih.gov/study.html?id='
    lnk = root_link + str(i)
    r = requests.get(lnk)
    if len(r.history)==0:
        with open('output/ndar_studies/study_'+str(i)+'.html', 'wb') as f:
            f.write(r.content)
    else:
        missing_ids.append(i)
    time.sleep(.2)

200
250
300
350
400
450
500
550
600
650
700
750


In [20]:
study_id = []
doi = []
studies = glob.glob('output/ndar_studies/*')
for study in studies:
    soup = BeautifulSoup(open(study), 'html.parser')
    study_id.append(re.findall('study_([0-9]+)', study))
    try:
        doi.append(soup.find_all('a', href=re.compile('dx.doi.org'))[0]['href'])
    except:
        doi.append('none')

In [52]:
study_info = pd.DataFrame({'doi':doi, 'study_id':[s[0] for s in study_id]})
study_info.head()
study_info = study_info[study_info.doi!='none']
study_info['doi2'] = study_info.doi.apply(lambda x: x[18:])
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}
response_list = []
for number, i in enumerate(study_info.doi2.tolist()):
    root_link = 'http://api.crossref.org/works/'
    lnk = root_link + str(i)
    r = requests.get(lnk,headers=headers)
    response_list.append(r)
    print(number)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212


In [58]:
for i in response_list:
    if i.text != 'Resource not found.':
        print(i.text)

{"status":"ok","message-type":"work-list","message-version":"1.0.0","message":{"facets":{},"total-results":108433163,"items":[{"indexed":{"date-parts":[[2019,4,16]],"date-time":"2019-04-16T17:10:39Z","timestamp":1555434639829},"reference-count":37,"publisher":"Institution of Engineering and Technology (IET)","issue":"1","content-domain":{"domain":[],"crossmark-restriction":false},"short-container-title":["IEE Proc., Electr. Power Appl."],"published-print":{"date-parts":[[2006]]},"DOI":"10.1049\/ip-epa:20050215","type":"journal-article","created":{"date-parts":[[2006,1,27]],"date-time":"2006-01-27T23:05:08Z","timestamp":1138403108000},"page":"129","source":"Crossref","is-referenced-by-count":60,"title":["Dual three-phase induction motor drive with digital current control in the stationary reference frame"],"prefix":"10.1049","volume":"153","author":[{"given":"R.","family":"Bojoi","sequence":"first","affiliation":[]},{"given":"E.","family":"Levi","sequence":"additional","affiliation":[]}

In [28]:
df_pmcid = pd.read_csv('data/PMC-ids.csv')

/home/riddleta/ac_knowl/ac_knowl/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
study_info.head()

,doi,study_id,doi2
1,http://dx.doi.org/10.15154/1503887,695,10.15154/1503887
3,http://dx.doi.org/10.15154/1343428,435,10.15154/1343428
5,http://dx.doi.org/10.15154/1240724,395,10.15154/1240724
8,http://dx.doi.org/10.15154/1358136,556,10.15154/1358136
11,http://dx.doi.org/10.15154/1163545,318,10.15154/1163545
